In [1]:
import numpy as np
import pandas as pd

In [3]:
title_basics = pd.read_csv('data/title.basics.tsv', delimiter='\t', encoding='utf-8')

/Users/huangzhisheng/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
title_basics_movie = title_basics[title_basics['titleType'] == 'movie']

#print title_basics_movie.head(10)
print "Number of movies", title_basics_movie.shape[0]

Number of movies 478229


In [5]:
title_basics_movie_genres = title_basics_movie[title_basics_movie.genres != '\N']
uid = title_basics_movie_genres['tconst']
print title_basics_movie_genres.head(10)
#print uid[0:19]
print "Number of movies with genre values:", title_basics_movie_genres.shape[0]


        tconst titleType                     primaryTitle  \
8    tt0000009     movie                       Miss Jerry   
145  tt0000147     movie    The Corbett-Fitzsimmons Fight   
332  tt0000335     movie            Soldiers of the Cross   
571  tt0000574     movie      The Story of the Kelly Gang   
611  tt0000615     movie               Robbery Under Arms   
626  tt0000630     movie                           Hamlet   
669  tt0000675     movie                      Don Quijote   
670  tt0000676     movie  Don Álvaro o la fuerza del sino   
673  tt0000679     movie   The Fairylogue and Radio-Plays   
730  tt0000739     movie        El pastorcito de Torrente   

                       originalTitle  isAdult startYear endYear  \
8                         Miss Jerry        0      1894      \N   
145    The Corbett-Fitzsimmons Fight        0      1897      \N   
332            Soldiers of the Cross        0      1900      \N   
571      The Story of the Kelly Gang        0      1906     

In [6]:
# genres contains all the genres in the dataset
genres = title_basics_movie_genres[['genres']]
print genres.head(10)
#print type(genres)
#print type(title_basics_movie_genres)

                     genres
8                   Romance
145  Documentary,News,Sport
332         Biography,Drama
571   Biography,Crime,Drama
611                   Drama
626                   Drama
669                   Drama
670                   Drama
673       Adventure,Fantasy
730                   Drama


In [60]:
movies_per_genres = genres.groupby(['genres'])['genres'].count()
print movies_per_genres[0:50]
print "Numbers of genres", movies_per_genres.shape[0]

genres
Action                          10322
Action,Adult                        8
Action,Adult,Adventure              1
Action,Adult,Comedy                 7
Action,Adult,Crime                  9
Action,Adult,Documentary            1
Action,Adult,Drama                  4
Action,Adult,Fantasy                1
Action,Adult,Horror                 1
Action,Adult,Western                1
Action,Adventure                 1020
Action,Adventure,Animation        343
Action,Adventure,Biography         75
Action,Adventure,Comedy           939
Action,Adventure,Crime            576
Action,Adventure,Documentary      194
Action,Adventure,Drama           1181
Action,Adventure,Family           138
Action,Adventure,Fantasy          484
Action,Adventure,Film-Noir          1
Action,Adventure,History           94
Action,Adventure,Horror           119
Action,Adventure,Music             69
Action,Adventure,Musical            9
Action,Adventure,Mystery           96
Action,Adventure,Romance          297
Actio